<a href="https://colab.research.google.com/github/alinatl/demo-spanish-bot/blob/main/dataset_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import login

login()

In [ ]:
!pip install transformers datasets trl huggingface_hub

In [ ]:
from datasets import load_dataset, DatasetDict
from datasets import Dataset

df = load_dataset(path="bavard/personachat_truecased", split="train").to_pandas()
filtered_df = df.loc[df.groupby("conv_id")["utterance_idx"].idxmax()]

In [ ]:
history_chats = [list(x['history']) for i, x in filtered_df.iterrows()]
dataset = Dataset.from_dict({"en": history_chats}).select(range(1000))
dataset

Dataset({
    features: ['en'],
    num_rows: 1000
})

In [ ]:
dataset['en'][0]

["Hi, how are you doing? I'm getting ready to do some cheetah chasing to stay in shape.",
 'You must be very fast. Hunting is one of my favorite hobbies.',
 'I am! For my hobby I like to do canning or some whittling.',
 'I also remodel homes when I am not out bow hunting.',
 "That's neat. When I was in high school I placed 6th in 100m dash!",
 "That's awesome. Do you have a favorite season or time of year?",
 'I do not. But I do have a favorite meat since that is all I eat exclusively.',
 'What is your favorite meat to eat?',
 'I would have to say its prime rib. Do you have any favorite foods?',
 'I like chicken or macaroni and cheese.',
 'Do you have anything planned for today? I think I am going to do some canning.',
 'I am going to watch football. What are you canning?',
 'I think I will can some jam. Do you also play footfall for fun?']

In [ ]:
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-es", device=0)

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/466M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/804k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/824k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


In [1]:
def translate_example(example):
    roles = ['user', 'assistant']
    translation = translator(example["en"])

    dialogue = []
    for idx, text in enumerate(example["en"]):
      role = roles[idx % 2]
      dialogue.append({'role': role, 'content': text})
    example["en"] = dialogue

    dialogue = []
    for idx, text in enumerate(translation):
      role = roles[idx % 2]
      dialogue.append({'role': role, 'content': text['translation_text']})
    example["sp"] = dialogue
    return example


translated_dataset = dataset.map(translate_example)

data_train_valid = translated_dataset.train_test_split(test_size=0.80)

In [ ]:
split_dataset = DatasetDict({
    "train": data_train_valid['test'],
    "valid": data_train_valid['train'],
})

split_dataset.push_to_hub("alinatl/personachat_truecased-spanish", private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/543 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/alinatl/personachat_truecased-spanish/commit/b9af2036b86cc9f171f0fb0069192a2855eead23', commit_message='Upload dataset', commit_description='', oid='b9af2036b86cc9f171f0fb0069192a2855eead23', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/alinatl/personachat_truecased-spanish', endpoint='https://huggingface.co', repo_type='dataset', repo_id='alinatl/personachat_truecased-spanish'), pr_revision=None, pr_num=None)